### Projeto Final de Spark
#### Campanha Nacional de Vacinação contra Covid-19

Autor: Arthur Freitas Gomes

Turma: Semantix Academy - Big Data Engineer 04-21

Professor: Rodrigo Rebouças

#### Enviar os dados para o hdfs

No terminal:

agomes:~/spark/input$ sudo mkdir desafio

agomes:~/spark/input/desafio$ sudo curl -O https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

agome:~/spark/input/desafio$ sudo unrar x 04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

agomes:~/spark/input/desafio$ docker exec -it namenode bash

    root@namenode:/# hdfs dfs -mkdir -p /user/arthur/desafio
    root@namenode:/# hdfs dfs -put /input/desafio/* /user/arthur/desafio
    root@namenode:/# hdfs dfs -ls /user/arthur/desafio

In [4]:
!hdfs dfs -ls /user/arthur/desafio

Found 5 items
-rw-r--r--   3 root supergroup   62492959 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup   76520681 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup   91120916 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup    3046774 2021-07-08 09:17 /user/arthur/desafio/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv
drwxr-xr-x   - root supergroup          0 2021-07-08 22:17 /user/arthur/desafio/hive


In [5]:
!hdfs dfs -rm /user/arthur/desafio/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar

rm: `/user/arthur/desafio/04bd3419b22b9cc5c6efac2c6528100d_HIST_PAINEL_COVIDBR_06jul2021.rar': No such file or directory


#### Otimizar todos os dados do hdfs para uma tabela Hive particionada por município

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [80]:
covid = spark.read.csv("/user/arthur/desafio/*.csv", header='true',sep=';')

In [81]:
covid = covid.na.fill({'codmun': 0})

In [82]:
#covid.select(isnan('municipio')).filter("isnan(municipio) = true").show()

In [83]:
covid = covid.withColumnRenamed("interior/metropolitana", "interiorOUmetropolitana")
covid = covid.withColumn("col_partition", col("coduf"))

In [84]:
covid.describe()

DataFrame[summary: string, regiao: string, estado: string, municipio: string, coduf: string, codmun: string, codRegiaoSaude: string, nomeRegiaoSaude: string, data: string, semanaEpi: string, populacaoTCU2019: string, casosAcumulado: string, casosNovos: string, obitosAcumulado: string, obitosNovos: string, Recuperadosnovos: string, emAcompanhamentoNovos: string, interiorOUmetropolitana: string, col_partition: string]

In [85]:
#!hdfs dfs -rm -r /user/arthur/desafio/covid_spark

rm: `/user/arthur/desafio/covid_spark': No such file or directory


In [86]:
covid.write.csv('/user/arthur/desafio/covid_spark', header='true')

In [87]:
!hdfs dfs -ls /user/arthur/desafio/covid_spark

Found 13 items
-rw-r--r--   2 root supergroup          0 2021-07-09 00:00 /user/arthur/desafio/covid_spark/_SUCCESS
-rw-r--r--   2 root supergroup   22275751 2021-07-09 00:00 /user/arthur/desafio/covid_spark/part-00000-c1a9be0b-4525-4360-88d4-12f19acbcf1b-c000.csv
-rw-r--r--   2 root supergroup   22282685 2021-07-09 00:00 /user/arthur/desafio/covid_spark/part-00001-c1a9be0b-4525-4360-88d4-12f19acbcf1b-c000.csv
-rw-r--r--   2 root supergroup   22286237 2021-07-09 00:00 /user/arthur/desafio/covid_spark/part-00002-c1a9be0b-4525-4360-88d4-12f19acbcf1b-c000.csv
-rw-r--r--   2 root supergroup   22238741 2021-07-09 00:00 /user/arthur/desafio/covid_spark/part-00003-c1a9be0b-4525-4360-88d4-12f19acbcf1b-c000.csv
-rw-r--r--   2 root supergroup   22257577 2021-07-09 00:00 /user/arthur/desafio/covid_spark/part-00004-c1a9be0b-4525-4360-88d4-12f19acbcf1b-c000.csv
-rw-r--r--   2 root supergroup   22250891 2021-07-09 00:00 /user/arthur/desafio/covid_spark/part-00005-c1a9be0b-4525-4360-88d4-12f19acbcf1b

In [88]:
!hdfs dfs -cat /user/arthur/desafio/covid_spark/part-00011-c1a9be0b-4525-4360-88d4-12f19acbcf1b-c000.csv

regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interiorOUmetropolitana,col_partition
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-23,25,12149,513,2,11,0,"","",0,50
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-24,25,12149,515,2,11,0,"","",0,50
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-25,25,12149,519,4,11,0,"","",0,50
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-26,25,12149,519,0,11,0,"","",0,50
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-27,26,12149,523,4,11,0,"","",0,50
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-28,26,12149,523,0,11,0,"","",0,50
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-29,26,12149,526,3,11,0,"","",0,50
Centro-Oeste,MS,Aral Moreira,50,500124,50003,DOURADOS,2021-06-30,26,12149,526,0,11,0,"","",0,50
C

Centro-Oeste,MS,Corguinho,50,500310,50001,CAMPO GRANDE,2021-06-30,26,5947,243,1,9,1,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-01,53,15253,297,4,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-02,53,15253,297,0,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-03,1,15253,297,0,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-04,1,15253,298,1,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-05,1,15253,301,3,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-06,1,15253,305,4,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-07,1,15253,309,4,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-08,1,15253,316,7,5,0,"","",0,50
Centro-Oeste,MS,Coronel Sapucaia,50,500315,50003,DOURADOS,2021-01-09,1,15253,318,2,5,0,"","",0,50
Centro-Oeste,MS,Coron

Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-15,19,47083,4201,22,60,1,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-16,20,47083,4232,31,60,0,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-17,20,47083,4251,19,60,0,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-18,20,47083,4271,20,61,1,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-19,20,47083,4325,54,62,1,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-20,20,47083,4370,45,62,0,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-21,20,47083,4415,45,62,0,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-22,20,47083,4456,41,63,1,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-23,21,47083,4510,54,64,1,"","",0,50
Centro-Oeste,MS,Maracaju,50,500540,50001,CAMPO GRANDE,2021-05-24,21,47083,4540,30,64,0,"","",0,50
Centro-Oeste,MS,Mara

Centro-Oeste,MS,Selvíria,50,500780,50004,TRES LAGOAS,2021-06-29,26,6529,536,-1,13,0,"","",0,50
Centro-Oeste,MS,Selvíria,50,500780,50004,TRES LAGOAS,2021-06-30,26,6529,536,0,13,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-01,53,57665,2495,0,31,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-02,53,57665,2497,2,31,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-03,1,57665,2498,1,31,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-04,1,57665,2498,0,31,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-05,1,57665,2511,13,31,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-06,1,57665,2521,10,31,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-07,1,57665,2537,16,31,0,"","",0,50
Centro-Oeste,MS,Sidrolândia,50,500790,50001,CAMPO GRANDE,2021-01-08,1,57665,2548,11,31,0,"","",0,50
Centro-Oeste,

Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-10,23,34966,2795,24,66,0,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-11,23,34966,2804,9,66,0,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-12,23,34966,2806,2,67,1,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-13,24,34966,2807,1,68,1,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-14,24,34966,2814,7,69,1,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-15,24,34966,2857,43,69,0,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-16,24,34966,2866,9,69,0,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MATOGROSSENSE,2021-06-17,24,34966,2873,7,69,0,"","",0,51
Centro-Oeste,MT,Barra do Bugres,51,510170,51007,MEDIO NORTE MA

Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-15,24,612547,89965,393,2969,8,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-16,24,612547,90308,343,2971,2,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-17,24,612547,90610,302,2977,6,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-18,24,612547,90916,306,2983,6,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-19,24,612547,91024,108,2987,4,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-20,25,612547,91083,59,2993,6,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-21,25,612547,91319,236,2996,3,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-22,25,612547,91594,275,3001,5,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA,2021-06-23,25,612547,91878,284,3013,12,"","",1,51
Centro-Oeste,MT,Cuiabá,51,510340,51002,BAIXADA CUIABANA

Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-04,9,11221,319,0,18,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-05,9,11221,319,0,18,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-06,9,11221,319,0,18,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-07,10,11221,320,1,18,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-08,10,11221,321,1,18,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-09,10,11221,323,2,19,1,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-10,10,11221,323,0,19,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-11,10,11221,323,0,19,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-12,10,11221,323,0,19,0,"","",0,51
Centro-Oeste,MT,Juscimeira,51,510520,51013,SUL MATOGROSSENSE,2021-03-13,10,1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



No terminal

agomes:~/spark$ docker exec -it hive-server bash

    root@hive_server:/opt# beeline -u jdbc:hive2://localhost:10000
    
    0: jdbc:hive2://localhost:10000> create database arthur;
    0: jdbc:hive2://localhost:10000> show databases;
    +----------------+
    | database_name  |
    +----------------+
    | arthur         |
    | default        |
    +----------------+

    0: jdbc:hive2://localhost:10000> use arthur;
       
    0: jdbc:hive2://localhost:10000> create external table covid(regiao string,estado string,municipio string,coduf int,codmun int,codRegiaoSaude int,nomeRegiaoSaude string,data string,semanaEpi int,populacaoTCU2019 int,casosAcumulado int,casosNovos int,obitosAcumulado int,obitosNovos int,Recuperadosnovos int,emAcompanhamentoNovos int,interiorOUmetropolitana string, col_partition int) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile location 'hdfs://namenode:8020/user/arthur/desafio/hive' tblproperties("skip.header.line.count"="1");
    
    #Para ver estrutura
    0: jdbc:hive2://localhost:10000> desc formatted covid; 
    
    0: jdbc:hive2://localhost:10000> load data inpath '/user/arthur/desafio/covid_spark/*.csv' overwrite into table covid;
    
    0: jdbc:hive2://localhost:10000> select * from covid limit 10;
    
    #Para permitir Particionamento Dinâmico
    0: jdbc:hive2://localhost:10000> SET hive.exec.dynamic.partition = true;

    0: jdbc:hive2://localhost:10000> SET hive.exec.dynamic.partition.mode = nonstrict;
    
    0: jdbc:hive2://localhost:10000> create external table covid_otm(regiao string,estado string, municipio string,coduf int,codmun int,codRegiaoSaude int,nomeRegiaoSaude string,data string,semanaEpi int,populacaoTCU2019 int,casosAcumulado int,casosNovos int,obitosAcumulado int,obitosNovos int,Recuperadosnovos int,emAcompanhamentoNovos int,interiorOUmetropolitana string) partitioned by (col_partition int) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile location 'hdfs://namenode:8020/user/arthur/desafio/hive/otm' tblproperties("skip.header.line.count"="1");
    
    0: jdbc:hive2://localhost:10000> insert overwrite table covid_otm partition (col_partition) select * from covid;


In [89]:
!hdfs dfs -ls /user/arthur/desafio/hive/otm

Found 30 items
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=0
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=1
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=11
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=12
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=13
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=14
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=15
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=16
drwxr-xr-x   - root supergroup          0 2021-07-09 00:02 /user/arthur/desafio/hive/otm/col_partition=17
drwxr-xr-x   - root supergroup   

create external table covid(regiao string,estado string,coduf int,codmun int,codRegiaoSaude int,nomeRegiaoSaude string,data string,semanaEpi int,populacaoTCU2019 int,casosAcumulado int,casosNovos int,obitosAcumulado int,obitosNovos int,Recuperadosnovos int,emAcompanhamentoNovos int,interiorOUmetropolitana string) partitioned by (municipio string) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile location 'hdfs://namenode:8020/user/arthur/desafio/hive/otm' tblproperties("skip.header.line.count"="1");

load data inpath '/user/arthur/desafio/covid_spark/*.csv' overwrite into table covid partition(municipio);

create external table covid_otm(regiao string,estado string,coduf int,codmun int,codRegiaoSaude int,nomeRegiaoSaude string,data string,semanaEpi int,populacaoTCU2019 int,casosAcumulado int,casosNovos int,obitosAcumulado int,obitosNovos int,Recuperadosnovos int,emAcompanhamentoNovos int,interiorOUmetropolitana string) partitioned by (municipio string) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile location 'hdfs://namenode:8020/user/arthur/desafio/hive/otm' tblproperties("skip.header.line.count"="1");

insert overwrite table covid_otm partition (municipio) select * from covid;
    